In [1]:
import kfp
import kfp.components as comp
import kfp.compiler as pipe_comp
import robin_stocks as rs
import pandas as pd
import requests
import numpy as np
import nbimporter
from typing import NamedTuple
from collections import namedtuple
from google.cloud import storage
from datetime import timedelta, datetime
import time
import csv
import dash
from tqdm import tqdm

In [2]:
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np

In [3]:
#include functions from notebooks
%run auth.ipynb
%run recent_stock_news.ipynb
%run check_volume.ipynb
%run get_trending_stocks.ipynb
%run purchase_stock.ipynb
%run tradeable_stocks.ipynb
%run cloud_CSV_storage.ipynb


In [4]:
#making notebook functions available
import auth
import recent_stock_news
import check_volume
import get_trending_stocks
import purchase_stock
import tradeable_stocks
import cloud_CSV_storage

Importing Jupyter notebook from auth.ipynb
Importing Jupyter notebook from recent_stock_news.ipynb
Importing Jupyter notebook from check_volume.ipynb
Importing Jupyter notebook from get_trending_stocks.ipynb
Importing Jupyter notebook from purchase_stock.ipynb
Importing Jupyter notebook from tradeable_stocks.ipynb
Importing Jupyter notebook from cloud_CSV_storage.ipynb


In [5]:
#shutdown worked to clear cache 
#I think the solution is a try get_header
#except we need to shutdown the kernal and rerun all the cells
get_header = rh_auth(username='', password='')

In [6]:
all_tradeable_stocks = get_tradeable_stocks()

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
Loading page 31 ...
Loading page 32 ...
Loading page 33 ...
Loading page 34 ...
Loading page 35 ...
Loading page 36 ...
Loading page 37 ...
Loading page 38 ...
Loading page 39 ...
Loading page 40 ...
Loading page 41 ...
Loading page 42 ...
Loading page 43 ...
Loading page 44 ...
Loading page 45 ...
Loading page 46 ...
Loading page 47 ...
Loading page 48 ...
Loading page 49 ...
Loading page 50 ...
Load

In [ ]:
current_time = datetime.now()
current_time_form = (current_time.hour * 100) + current_time.minute
while current_time_form >= 600 and current_time_form <= 1230:
    #push purchase list to cloud
    print(current_time.hour, current_time.minute)
    
    #should try push_stock_purchase_list_to_cloud
    #expect shutdown and rerun all the cells
    #likely due to a logging in/authentication issue
    push_stock_purchase_list_to_cloud(all_tradeable_stocks)
    purchase_list = get_purchase_list_from_cloud()
    if len(purchase_list) > 0:
        buy_stock(purchase_list)

    current_time = datetime.now()
    print(current_time.hour, current_time.minute)
    current_time_form = (current_time.hour * 100) + current_time.minute

9 40
filtered for recent news
filtered for volume
[['NSANY', "Exclusive: Who's in charge at Nissan? COO's allies push to give him shared CEO role – sources", 1.001394268009295]]
pushed to cloud
32.580000000001746 8.08
next stock for purchase:  NSANY
this is my buy_power: 32.580000000001746
This is the price of the stock: 8.08
setting sell price
{'id': '20a8671d-b2b8-4e85-a631-58f8e8f2f9fa', 'ref_id': '067205a3-e600-48b9-9b9b-2e76866c7a0a', 'url': 'https://api.robinhood.com/orders/20a8671d-b2b8-4e85-a631-58f8e8f2f9fa/', 'account': 'https://api.robinhood.com/accounts/5QV61626/', 'position': 'https://api.robinhood.com/positions/5QV61626/d6b76732-f3ab-482a-8535-a4574b52375c/', 'cancel': 'https://api.robinhood.com/orders/20a8671d-b2b8-4e85-a631-58f8e8f2f9fa/cancel/', 'instrument': 'https://api.robinhood.com/instruments/d6b76732-f3ab-482a-8535-a4574b52375c/', 'cumulative_quantity': '0.00000000', 'average_price': None, 'fees': '0.00', 'state': 'unconfirmed', 'type': 'limit', 'side': 'sell', '